In [1]:
from pathlib import Path
import pickle

import jax
from jax import jit, device_put
import jax.numpy as jnp
from jax import random
from tqdm import tqdm
import optax
from numpyro.infer import SVI, Trace_ELBO
import numpy as np
import matplotlib.pyplot as plt

from src.models.CCVAE import CCVAE
from src.models.encoder_decoder import MNISTEncoder, MNISTDecoder, CIFAR10Encoder, CIFAR10Decoder
from src.data_loading.loaders import get_data_loaders
from src.losses import CCVAE_ELBO


# Set up random seed
seed = 42

# DATASET
dataset_name = "CELEBA" # use "CIFAR10"*
#dataset_name = 'CIFAR10'

encoder_class = MNISTEncoder if dataset_name=="MNIST" else CIFAR10Encoder
decoder_class = MNISTDecoder if dataset_name=="MNIST" else CIFAR10Decoder
distribution = "bernoulli" if dataset_name=="MNIST" else "laplace"

# Data loading

img_shape, loader_dict, size_dict = get_data_loaders(dataset_name=dataset_name, 
                                          p_test=0.2, 
                                          p_val=0.2, 
                                          p_supervised=0.9, 
                                          batch_size=128, 
                                          num_workers=6, 
                                          seed=seed)

scale_factor = 0.1 * size_dict["supervised"] # IMPORTANT, maybe run a grid search (0.3 on cifar)

# Set up model
ccvae = CCVAE(encoder_class, 
               decoder_class, 
               10, 
               50, 
               img_shape, 
               scale_factor=scale_factor, 
               distribution=distribution,
               multiclass=False
)
print("Model set up!")

# Set up two learning rate schedules, one for the VAE and one for the classifier
lr_schedule_vae = optax.piecewise_constant_schedule(
    init_value=2e-3,
    boundaries_and_scales={
        20 * len(loader_dict["semi_supervised"]): 0.7,
    }
)
lr_schedule_classifier = optax.piecewise_constant_schedule(
    init_value=1e-3,
    boundaries_and_scales={
        20 * len(loader_dict["semi_supervised"]): 0.7,
    }
)
# Set up two optimizers, one for the VAE and one for the classifier

optimizer_vae = optax.adam(lr_schedule_vae)
optimizer_classifier = optax.adamw(lr_schedule_classifier, weight_decay=3e-4)

/users/eleves-a/2020/david.heurtel-depeiges/Desktop/CCVAE_Jax/pgm/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully loaded MNIST dataset.
Total num samples 60000
Num test samples: 12000
Num validation samples: 9600
Num supervised samples: 34560
Num unsupervised samples: 3840
Model set up!


In [2]:
import flax
def flattened_traversal(fn):
  """Returns function that is called with `(path, param)` instead of pytree."""
  def mask(tree):
    flat = flax.traverse_util.flatten_dict(tree)
    return flax.traverse_util.unflatten_dict(
        {k: fn(k, v) for k, v in flat.items()})
  return mask

In [3]:
vae_classifier_fn = lambda path, param: 'low' if "Classifier" in path else 'high'

vae_classifier_fn = flattened_traversal(vae_classifier_fn)

# Set up optimizer

optimizer = optax.multi_transform({'low': optimizer_classifier, 'high': optimizer_vae}, vae_classifier_fn)

In [4]:

# Set up SVI
svi_supervised = SVI(ccvae.model_supervised, 
            ccvae.guide_supervised, 
            optim=optimizer, 
            loss=CCVAE_ELBO()
)

svi_unsupervised = SVI(ccvae.model_unsupervised, 
            ccvae.guide_unsupervised,
            optim=optimizer, 
            loss=Trace_ELBO()
)

state = svi_supervised.init(
    random.PRNGKey(seed), 
    xs=jnp.ones((1,)+img_shape), 
    ys=jnp.ones((1), dtype=jnp.int32)
)
svi_unsupervised.init(
    random.PRNGKey(seed), 
    xs=jnp.ones((1,)+img_shape)
)
print("SVI set up!")


# Train functions
@jit
def train_step_supervised(state, batch):
    x, y = batch
    state, loss_supervised = svi_supervised.update(state, xs=x, ys=y)
    
    return state, loss_supervised

@jit
def train_step_unsupervised(state, batch):
    x = batch
    state, loss_unsupervised = svi_unsupervised.update(state, xs=x)
    
    return state, loss_unsupervised

# Training
semi_supervised_loader = loader_dict["semi_supervised"]
validation_loader = loader_dict["validation"]
test_loader = loader_dict["test"]

print("Start training.")
loss_rec_supervised = []
loss_rec_unsupervised = []
validation_accuracy_rec = []


SVI set up!
Start training.


/tmp/ipykernel_302475/815935608.py:19: UserWarning: Currently, SVI with Trace_ELBO loss does not support models with discrete latent variables
  svi_unsupervised.init(


In [5]:
num_epochs = 100
for epoch in tqdm(range(1, num_epochs + 1)):
    running_loss = 0.0

    loss_rec_step_supervised = []
    loss_rec_step_unsupervised = []

    # Trainning
    for is_supervised, batch in semi_supervised_loader: 
        batch = device_put(batch)

        if is_supervised:
            state, loss_supervised = train_step_supervised(state, batch)
            loss_rec_step_supervised.append(loss_supervised)
        else:
            state, loss_unsupervised = train_step_unsupervised(state, batch)
            loss_rec_step_unsupervised.append(loss_unsupervised)
    
    loss_epoch_supervised = np.mean(loss_rec_step_supervised)
    loss_epoch_unsupervised = np.mean(loss_rec_step_unsupervised)

    loss_rec_supervised.append(loss_epoch_supervised)
    loss_rec_unsupervised.append(loss_epoch_unsupervised)
    
    validation_accuracy = 0.0

    for batch in validation_loader:
        batch = device_put(batch)
        x, y = batch
        ypred = ccvae.classify(state[0][1][0], x)
        validation_accuracy += jnp.mean(y == ypred)
    
    validation_accuracy /= len(validation_loader)
    validation_accuracy_rec.append(validation_accuracy)
    
    print("\nEpoch:", 
          epoch, 
          "loss sup:", 
          loss_epoch_supervised, 
          "loss unsup:", 
          loss_epoch_unsupervised, 
          "val acc:", 
          validation_accuracy
    )

print("Training finished!")

  1%|          | 1/100 [00:09<16:12,  9.82s/it]


Epoch: 1 loss sup: 34727.152 loss unsup: 33238.176 val acc: 0.13854165


  2%|▏         | 2/100 [00:12<09:28,  5.80s/it]


Epoch: 2 loss sup: 25699.863 loss unsup: 25543.617 val acc: 0.19864583


  3%|▎         | 3/100 [00:15<07:11,  4.44s/it]

...


Epoch: 98 loss sup: 15522.7705 loss unsup: 15265.436 val acc: 0.15906249


 99%|█████████▉| 99/100 [04:48<00:02,  2.84s/it]


Epoch: 99 loss sup: 15494.393 loss unsup: 15245.385 val acc: 0.16645832


100%|██████████| 100/100 [04:51<00:00,  2.92s/it]


Epoch: 100 loss sup: 15503.29 loss unsup: 15239.277 val acc: 0.16854165
Training finished!
